# Evolver Loop 1 Analysis

## Goal: Understand pre-optimized submissions and establish true baseline

The evaluator correctly identified that we started from the wrong place. Let's score the pre-optimized submissions from top kernels.

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree

getcontext().prec = 30

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(0), float(tip_y)),
            (float(top_w / 2), float(tier_1_y)),
            (float(top_w / 4), float(tier_1_y)),
            (float(mid_w / 2), float(tier_2_y)),
            (float(mid_w / 4), float(tier_2_y)),
            (float(base_w / 2), float(base_y)),
            (float(trunk_w / 2), float(base_y)),
            (float(trunk_w / 2), float(trunk_bottom_y)),
            (float(-trunk_w / 2), float(trunk_bottom_y)),
            (float(-trunk_w / 2), float(base_y)),
            (float(-base_w / 2), float(base_y)),
            (float(-mid_w / 4), float(tier_2_y)),
            (float(-mid_w / 2), float(tier_2_y)),
            (float(-top_w / 4), float(tier_1_y)),
            (float(-top_w / 2), float(tier_1_y)),
        ])

        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

print('ChristmasTree class defined')

In [ ]:
def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    subset = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in subset.iterrows():
        x = str(row['x']).lstrip('s')
        y = str(row['y']).lstrip('s')
        deg = str(row['deg']).lstrip('s')
        trees.append(ChristmasTree(x, y, deg))
    return trees

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i:
                if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                    intersection = poly.intersection(polygons[idx])
                    if intersection.area > 1e-12:
                        return True
    return False

def get_bounding_box_side(trees):
    if not trees:
        return 0
    all_coords = []
    for tree in trees:
        coords = np.array(tree.polygon.exterior.coords)
        all_coords.append(coords)
    all_coords = np.vstack(all_coords)
    x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
    y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
    return max(x_range, y_range)

def score_submission(df, max_n=200):
    total_score = 0
    overlaps = []
    for n in range(1, max_n + 1):
        trees = load_trees_for_n(df, n)
        if len(trees) != n:
            print(f"Warning: n={n} has {len(trees)} trees instead of {n}")
            continue
        if has_overlap(trees):
            overlaps.append(n)
        side = get_bounding_box_side(trees)
        score_n = (side ** 2) / n
        total_score += score_n
    return total_score, overlaps

print('Scoring functions defined')

In [ ]:
# Score the pre-optimized submissions
print('Scoring pre-optimized submissions...')
print('='*60)

# bucket-of-chump submission
df_bucket = pd.read_csv('/home/code/preoptimized/submission.csv')
print(f'bucket-of-chump shape: {df_bucket.shape}')
bucket_score, bucket_overlaps = score_submission(df_bucket)
print(f'bucket-of-chump score: {bucket_score:.6f}')
print(f'Overlaps: {bucket_overlaps}')
print()

# santa-2025.csv submission
df_saspav = pd.read_csv('/home/code/preoptimized/santa-2025.csv')
print(f'santa-2025.csv shape: {df_saspav.shape}')
saspav_score, saspav_overlaps = score_submission(df_saspav)
print(f'santa-2025.csv score: {saspav_score:.6f}')
print(f'Overlaps: {saspav_overlaps}')
print()

print('='*60)
print(f'Target score: 68.922808')
print(f'Best pre-optimized: {min(bucket_score, saspav_score):.6f}')
print(f'Gap to target: {min(bucket_score, saspav_score) - 68.922808:.6f}')

In [ ]:
# Analyze which submission is better for each N
print('\nPer-N analysis of best submission...')
print('='*60)

better_bucket = 0
better_saspav = 0
max_diff = 0
max_diff_n = 0

for n in range(1, 201):
    trees_b = load_trees_for_n(df_bucket, n)
    trees_s = load_trees_for_n(df_saspav, n)
    
    side_b = get_bounding_box_side(trees_b)
    side_s = get_bounding_box_side(trees_s)
    
    score_b = (side_b ** 2) / n
    score_s = (side_s ** 2) / n
    
    diff = abs(score_b - score_s)
    if diff > max_diff:
        max_diff = diff
        max_diff_n = n
    
    if score_b < score_s:
        better_bucket += 1
    elif score_s < score_b:
        better_saspav += 1

print(f'bucket-of-chump better for {better_bucket} N values')
print(f'santa-2025.csv better for {better_saspav} N values')
print(f'Max difference at N={max_diff_n}: {max_diff:.6f}')

In [ ]:
# Create ensemble - best of both for each N
print('\nCreating ensemble of best configurations...')

ensemble_rows = []
ensemble_score = 0

for n in range(1, 201):
    trees_b = load_trees_for_n(df_bucket, n)
    trees_s = load_trees_for_n(df_saspav, n)
    
    side_b = get_bounding_box_side(trees_b)
    side_s = get_bounding_box_side(trees_s)
    
    # Choose better one
    if side_b <= side_s:
        best_df = df_bucket
        best_side = side_b
    else:
        best_df = df_saspav
        best_side = side_s
    
    ensemble_score += (best_side ** 2) / n
    
    # Copy rows
    prefix = f"{n:03d}_"
    subset = best_df[best_df['id'].str.startswith(prefix)]
    ensemble_rows.append(subset)

df_ensemble = pd.concat(ensemble_rows, ignore_index=True)
print(f'Ensemble shape: {df_ensemble.shape}')
print(f'Ensemble score: {ensemble_score:.6f}')
print(f'Gap to target: {ensemble_score - 68.922808:.6f}')

In [ ]:
# Save ensemble as starting point
df_ensemble.to_csv('/home/code/preoptimized/ensemble.csv', index=False)
print('Saved ensemble to /home/code/preoptimized/ensemble.csv')